In [1]:
# will autoupdate any of the packages imported:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import pyclifford as pc
from pyclifford import utils, paulialg, stabilizer
from pyclifford.utils import acq, ipow, clifford_rotate, pauli_transform
from pyclifford.paulialg import Pauli, PauliList, PauliPolynomial
import qutip as qt

In [3]:
def decompose(g, gs, ps):
    '''  Decompose a pauli string to phase*destabilizers*stabilizers
    Parameters:
    g: int(2*N) - the binary vector of a pauli string
    gs: int(2*N,2*N) - the full tableau
    ps: int(2*N) - phase of stabilizer and destabilizer
    
    Returns:
    phase: int - phase in terms of imaginery power
    b: int(N) - binary encoding of destabilizer decomposition
    c: int(N) - binary encoding of stabilizer decomposition
    '''
    phase = 0
    tmp_p = np.zeros_like(g)
    N = gs.shape[0]//2
    b = np.zeros(N).astype(int)
    c = np.zeros(N).astype(int)
    for i in range(N):
        if acq(g,gs[i]): #anti-commute
            b[i] = 1
            phase = phase - ipow(tmp_p,gs[i+N]) + ps[i+N]
            tmp_p = (tmp_p+gs[i+N])%2
    for i in range(N):
        if acq(g,gs[i+N]): #anti-commute
            c[i] = 1
            phase = phase - ipow(tmp_p,gs[i]) + ps[i]
            tmp_p = (tmp_p+gs[i])%2
    return phase%4, tmp_p, b, c

The above function will decompose a pauli string into combination of destabilizer generators and stabilizer generators.

Let's see an example, if the stabilizer generators are $g_0=-ZZ$ and $g_1=XX$ and destabilizer generators are $d_0=IX$, $d_1=XI$.

In [4]:
gs = stabilizer.stabilizer_state("-ZZ","XX").gs
ps = stabilizer.stabilizer_state("-ZZ","XX").ps

In [5]:
gs

array([[0, 1, 0, 1],
       [1, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

In [6]:
ps

array([2, 0, 0, 0])

In [7]:
decompose(np.array([1,0,1,1]),gs,ps)

(1, array([1, 0, 1, 1]), array([0, 1]), array([1, 1]))

From the result, we see pauli string $XY$ can be decomposed as
$$XY = i^1 d_0^0 d_1^1 g_0^1 g_1^1=i(XI)(-ZZ)(XX)$$

The single-qubit $X$ and $Z$ are

In [8]:
# def list2num(lst):
#     return np.sum(2**np.arange(len(lst)) * np.array(list(reversed(lst))))

# def num2list(num, N):
#     assert num < 2**N
#     return np.array(list(map(int, bin(num)[2:].zfill(N))))

In [61]:
class GeneralStabilizerState(object):

    # def __init__(self, *args, **kwargs):
    def __init__(self, chi, gs, ps):
        self.chi = chi
        self.gs = gs
        self.ps = ps
        
    # def __repr__(self):
    #     ''' will only show active stabilizers, 
    #         to see the full stabilizer tableau, convert to PauliList by [:] '''
    #     subrepr = repr(self.stabilizers)
    #     if subrepr == '':
    #         return 'StabilizerState()'
    #     else:
    #         return 'StabilizerState(\n{})'.format(subrepr).replace('\n','\n  ')

    
    def copy(self):
        return self.chi.copy(), self.gs.copy(), self.ps.copy()

    def rotate_by(self, generator, mask=None):
        # perform Clifford transformation by Pauli generator (in-place)
        if mask is None:
            clifford_rotate(generator.g, generator.p, self.gs, self.ps)
        else:
            mask2 = numpy.repeat(mask,  2)
            self.gs[:,mask2], self.ps = clifford_rotate(
                generator.g, generator.p, self.gs[:,mask2], self.ps)
        return self
        
    def transform_by(self, generator, mask=None):
        # perform Clifford transformation by Clifford map (in-place)
        if mask is None:
            self.gs, self.ps = pauli_transform(self.gs, self.ps, 
                clifford_map.gs, clifford_map.ps)
        else:
            mask2 = numpy.repeat(mask, 2)
            self.gs[:,mask2], self.ps = pauli_transform(
                self.gs[:,mask2], self.ps, clifford_map.gs, clifford_map.ps)
        return self
    
    def pauli_chnl_evol(self, phi, pl, pr):
        '''Perform general Clifford channel evolution.
        
        Parameters:
        phi: [c1, c2, ...] - list of Pauli channel coefficients.
        pl:  [int (2*N), int (2*N), ...] - list of left-multiplying Pauli ops.
        pr:  [int (2*N), int (2*N), ...] - list of right-multiplying Pauli ops. 
        
        Returns:
        chi in-place modified.
        '''
        # pre-store alpha, b, c to avoid redundant call of decompose
        al, bl, cl = [], [], []
        ar, br, cr = [], [], []
        for pm, pn in zip(pl, pr):
            # print("pm: ",pm)
            # print("gs: ",self.gs)
            # print("ps: ", self.ps)
            am, _, bm, cm = decompose(pm, self.gs, self.ps) 
            an, _, bn, cn = decompose(pn, self.gs, self.ps)
            al.append(am)
            bl.append(bm)
            cl.append(cm)
            ar.append(an)
            br.append(bn)
            cr.append(cn)
        
        # update chi
        chinew = {}
        for idx in range(len(phi)):
            phimn = phi[idx]
            am, bm, cm = al[idx], bl[idx], cl[idx]
            an, bn, cn = ar[idx], br[idx], cr[idx]
            # print(phimn)
            for (i, j), chiij in self.chi.items():
                i, j = np.array(list(i)), np.array(list(j))
                # update i, j
                inew = (i + bm)%2
                jnew = (j + bn)%2
                # update chiij
                ipow = (am-an+2*(np.dot(i,cm)+np.dot(j,cn)))%4
                chiijnew = chiij * phimn * 1j**ipow
                keynew = (tuple(inew), tuple(jnew))
                if keynew in chinew:
                    chinew[keynew] += chiijnew
                else:
                    chinew[keynew] = chiijnew
                # print(chinew)
        self.chi = chinew
            
            
    def pauli_expect(self, obs):
        '''Evaluate expectation values of Pauli observables on the
generalized stabilizer state.
        
        Parameters:
        obs: observable, can be Pauli, PauliPolynomial, PauliList
        z: fugacity of operator weight
        
        Returns:
        out: output (depending on the type of obs)
            * Pauli: promote to PauliPolynomial
            * PauliPolynomial O: Tr(rho O z^|O|)
            * Paulilist [O_i]: [Tr(rho O_i z^|O_i|)]
        '''
        if isinstance(obs, Pauli):
            # cast Pauli to PauliPolynomial
            return self.pauli_expect(obs.as_polynomial())
        elif isinstance(obs, PauliPolynomial):
            # cast PauliPolynomial to PauliList
            xs = self.expect(PauliList(obs.gs, obs.ps))
            return numpy.sum(obs.cs * xs)
        elif isinstance(obs, PauliList):
            (L, Ng) = obs.gs.shape
            N = Ng//2
            xs = np.zeros(L, dtype=np.complex_) # expectation values
            pa = 0
            for k in range(L):
                a, _, b, c = decompose(obs.gs[k], self.gs, self.ps)
                for (i, j), chi in self.chi.items():
                    i, j = np.array(list(i)), np.array(list(j))
                    # print(f'i: {i.shape}, j: {j.shape}, b:{b.shape}')
                if np.all((i+j+b) == 0):
                    # ipow = (a + obs.ps[k] + 2*i@c) % 4
                    ipow = (a + obs.ps[k] + 2*np.dot(i, c)) % 4
                    xs[k] += chiij * 1j**ipow
            return xs 
    def to_qutip(self):
        n = self.gs.shape[1]//2
        paulis = [qt.qeye(2), qt.sigmax(), qt.sigmay(), qt.sigmaz()]
        identity = qt.tensor([qt.qeye(2) for i in range(n)])
        stab_state = identity
        for i in range(n):
            current_stabilizer = pc.paulialg.Pauli(g=self.gs[i],p=self.ps[i]).to_qutip()
            stab_state *= (identity+current_stabilizer)/2
            # print('current_stabilizer:',current_stabilizer, 'stab_state:', stab_state)
        state = 0
        for k, e in self.chi.items():
            left_coor = k[0]
            right_coor = k[1]
            # print('left_coor:',left_coor,'right_coor',right_coor)
            left_destabilizer = identity
            for l in range(len(left_coor)):
                if left_coor[l] == 1:
                    left_destabilizer *= pc.paulialg.Pauli(g=self.gs[l+n],p=self.ps[l+n]).to_qutip()
                    # print(f'{l}th left destabilizer: {left_destabilizer}')
            right_destabilizer = identity
            for r in range(len(right_coor)):
                if right_coor[r] == 1:
                    right_destabilizer *= pc.paulialg.Pauli(g=self.gs[r+n],p=self.ps[r+n]).to_qutip()
                    # print(f'{r}th right destabilizer: {left_destabilizer}')
            state += e * left_destabilizer * stab_state * right_destabilizer
        return state

T gate in terms of Pauli channels:
$$
T^\dagger\rho T = \cos(\pi/8)^2 I\rho I+ i \sin(\pi/8)\cos(\pi/8)Z\rho I - i\sin(\pi/8)\cos(\pi/8) I\rho Z + \sin(\pi/8)^2 Z\rho Z
$$

#### single-qubit GHZ

The stabilizer of single-qubit GHZ state is $X$, so destabilizer is $Z$.

In [62]:
ghz1 = pc.ghz_state(1)
state = GeneralStabilizerState({((0,),(0,)):1.0},ghz1.gs,ghz1.ps)
print("qutip:",state.to_qutip(),"chi:",state.chi)

qutip: Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]] chi: {((0,), (0,)): 1.0}


In [63]:
phi = np.array([np.cos(np.pi/8)**2, 1j*np.sin(np.pi/8)*np.cos(np.pi/8),\
                -1j*np.sin(np.pi/8)*np.cos(np.pi/8), np.sin(np.pi/8)**2], dtype=np.complex_)
pl = [
np.array([0,0]),np.array([0,1]),np.array([0,0]),np.array([0,1])
]
pr = [
np.array([0,0]),np.array([0,0]),np.array([0,1]),np.array([0,1])
]

state.pauli_chnl_evol(phi,pl,pr)
print('qutip:',state.to_qutip(),'chi:',state.chi)

qutip: Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5       +0.j         0.35355339+0.35355339j]
 [0.35355339-0.35355339j 0.5       +0.j        ]] chi: {((0,), (0,)): (0.8535533905932737+0j), ((1,), (0,)): 0.3535533905932738j, ((0,), (1,)): -0.3535533905932738j, ((1,), (1,)): (0.14644660940672624+0j)}


#### two-qubit GHZ

$$\rho=\begin{pmatrix}1/2 &  &  & 1/2\\
\\
\\
1/2 &  &  & 1/2
\end{pmatrix}$$

In [64]:
ghz2 = pc.ghz_state(2)
state = GeneralStabilizerState({((0,0),(0,0)):1.0},ghz2.gs,ghz2.ps)
print("qutip:\n",state.to_qutip(),"\n chi:",state.chi)

qutip:
 Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]] 
 chi: {((0, 0), (0, 0)): 1.0}


$$(T\otimes I)^\dagger \rho (T\otimes I)=\begin{pmatrix}1/2 &  &  & e^{i\pi/4}/2\\
\\
\\
e^{-i\pi/4}/2 &  &  & 1/2
\end{pmatrix}$$

In [65]:
phi = np.array([np.cos(np.pi/8)**2, 1j*np.sin(np.pi/8)*np.cos(np.pi/8),\
                -1j*np.sin(np.pi/8)*np.cos(np.pi/8), np.sin(np.pi/8)**2], dtype=np.complex_)
pl = [
np.array([0,0,0,0]),np.array([0,1,0,0]),np.array([0,0,0,0]),np.array([0,1,0,0])
]
pr = [
np.array([0,0,0,0]),np.array([0,0,0,0]),np.array([0,1,0,0]),np.array([0,1,0,0])
]

state.pauli_chnl_evol(phi,pl,pr)
print("qutip:\n",state.to_qutip(),"\n chi:",state.chi)

qutip:
 Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5       +0.j         0.        +0.j         0.        +0.j
  0.35355339+0.35355339j]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.35355339-0.35355339j 0.        +0.j         0.        +0.j
  0.5       +0.j        ]] 
 chi: {((0, 0), (0, 0)): (0.8535533905932737+0j), ((0, 1), (0, 0)): 0.3535533905932738j, ((0, 0), (0, 1)): -0.3535533905932738j, ((0, 1), (0, 1)): (0.14644660940672624+0j)}


#### single-qubit random

In [70]:
rdn1 = pc.random_clifford_state(1)
state = GeneralStabilizerState({((0,),(0,)):1.0},rdn1.gs,rdn1.ps)
print("map:",rdn1.to_map(),"qutip:",state.to_qutip(),"chi:",state.chi)

map: CliffordMap(
  X0-> +Z
  Z0-> +Y) qutip: Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5+0.j  0. -0.5j]
 [0. +0.5j 0.5+0.j ]] chi: {((0,), (0,)): 1.0}


So, stabilizer is $Y$, destabilizer is $Z$.

$$T^\dagger \rho T=\begin{pmatrix}1/2 & e^{-i\pi/4}/2\\
e^{i\pi/4}/2 & 1/2
\end{pmatrix}$$

In [71]:
phi = np.array([np.cos(np.pi/8)**2, 1j*np.sin(np.pi/8)*np.cos(np.pi/8),\
                -1j*np.sin(np.pi/8)*np.cos(np.pi/8), np.sin(np.pi/8)**2], dtype=np.complex_)
pl = [
np.array([0,0]),np.array([0,1]),np.array([0,0]),np.array([0,1])
]
pr = [
np.array([0,0]),np.array([0,0]),np.array([0,1]),np.array([0,1])
]

state.pauli_chnl_evol(phi,pl,pr)
print('qutip:',state.to_qutip(),'chi:',state.chi)

qutip: Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0.5       +0.j         0.35355339-0.35355339j]
 [0.35355339+0.35355339j 0.5       +0.j        ]] chi: {((0,), (0,)): (0.8535533905932737+0j), ((1,), (0,)): 0.3535533905932738j, ((0,), (1,)): -0.3535533905932738j, ((1,), (1,)): (0.14644660940672624+0j)}


#### two-qubit random

$$\rho=\frac{1}{4}\begin{pmatrix}1 & -i & -1 & -i\\
i & 1 & -i & 1\\
-1 & i & 1 & i\\
i & 1 & -i & 1
\end{pmatrix}$$

In [ ]:
rdn2 = pc.random_clifford_state(2)
state = GeneralStabilizerState({((0,0),(0,0)):1.0},rdn2.gs,rdn2.ps)
print("map:",rdn2.to_map(),"qutip:",state.to_qutip(),"chi:",state.chi)

map: CliffordMap(
  X0-> +XX
  Z0-> +YX
  X1-> +IX
  Z1-> +ZY) qutip: Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25+0.j    0.  -0.25j -0.25+0.j    0.  -0.25j]
 [ 0.  +0.25j  0.25+0.j    0.  -0.25j  0.25+0.j  ]
 [-0.25+0.j    0.  +0.25j  0.25+0.j    0.  +0.25j]
 [ 0.  +0.25j  0.25+0.j    0.  -0.25j  0.25+0.j  ]] chi: {((0, 0), (0, 0)): 1.0}


$$(T\otimes I)^\dagger \rho (T\otimes I)=\begin{pmatrix}0.25 & -0.25i & -0.176777-0.176777i & 0.176777-0.176777i\\
0.25i & 0.25 & 0.176777-0.176777i & 0.176777+0.176777i\\
-0.176777+0.176777i & 0.176777+0.176777i & 0.25 & 0.25i\\
0.176777+0.176777i & 0.176777-0.176777i & -0.25i & 0.25
\end{pmatrix}$$

In [75]:
phi = np.array([np.cos(np.pi/8)**2, 1j*np.sin(np.pi/8)*np.cos(np.pi/8),\
                -1j*np.sin(np.pi/8)*np.cos(np.pi/8), np.sin(np.pi/8)**2], dtype=np.complex_)
pl = [
np.array([0,0,0,0]),np.array([0,1,0,0]),np.array([0,0,0,0]),np.array([0,1,0,0])
]
pr = [
np.array([0,0,0,0]),np.array([0,0,0,0]),np.array([0,1,0,0]),np.array([0,1,0,0])
]

state.pauli_chnl_evol(phi,pl,pr)
print("qutip:\n",state.to_qutip(),"\n chi:",state.chi)

qutip:
 Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.25     +0.j         0.       -0.25j      -0.1767767-0.1767767j
   0.1767767-0.1767767j]
 [ 0.       +0.25j       0.25     +0.j         0.1767767-0.1767767j
   0.1767767+0.1767767j]
 [-0.1767767+0.1767767j  0.1767767+0.1767767j  0.25     +0.j
   0.       +0.25j     ]
 [ 0.1767767+0.1767767j  0.1767767-0.1767767j  0.       -0.25j
   0.25     +0.j       ]] 
 chi: {((0, 0), (0, 0)): (0.8535533905932737+0j), ((1, 0), (0, 0)): (0.3535533905932738-0j), ((0, 0), (1, 0)): (0.3535533905932738+0j), ((1, 0), (1, 0)): (0.14644660940672624+0j)}


### Test usage of generalized stabilizer

In [14]:
# creat generalized stabilizer
h1 = (0, 1)
h2 = (1, 0)
key = (h1, h2)
ele = 0.125
chi = {key: ele,((0,0),(0,0)):0.1}

ss = stabilizer.stabilizer_state("-ZZ","XX")
gs = ss.gs
ps = ss.ps
gss = GeneralStabilizerState(chi, gs, ps)
# (_, N2) = gs.shape
# N = N2//2
# ds = gs[N:2*N]

In [15]:
for k in chi.keys():
    print(k)

((0, 1), (1, 0))
((0, 0), (0, 0))


In [16]:
# test Clifford rotation

In [17]:
# test Pauli channel evolution
phi = np.array([0.25, 0.25, 0.25, 0.25], dtype=np.complex_)
pl = [np.array([0,0,0,1]),np.array([0,0,1,0]),np.array([0,1,0,0]),np.array([1,0,0,0])]
pr = [np.array([1,0,0,0]),np.array([0,1,0,0]),np.array([0,0,1,0]),np.array([0,0,0,1])]
 
gss.pauli_chnl_evol(phi, pl, pr)

pm:  [0 0 0 1]
gs:  [[0 1 0 1]
 [1 0 1 0]
 [0 0 1 0]
 [0 1 0 0]]
ps:  [2 0 0 0]
pm:  [0 0 1 0]
gs:  [[0 1 0 1]
 [1 0 1 0]
 [0 0 1 0]
 [0 1 0 0]]
ps:  [2 0 0 0]
pm:  [0 1 0 0]
gs:  [[0 1 0 1]
 [1 0 1 0]
 [0 0 1 0]
 [0 1 0 0]]
ps:  [2 0 0 0]
pm:  [1 0 0 0]
gs:  [[0 1 0 1]
 [1 0 1 0]
 [0 0 1 0]
 [0 1 0 0]]
ps:  [2 0 0 0]


In [18]:
gss.chi

{((0, 1), (1, 0)): (0.00048828125-0j), ((0, 0), (0, 0)): (0.000390625-0j)}

In [19]:
testList = np.zeros(4)

In [20]:
testList

array([0., 0., 0., 0.])

In [21]:
# test Pauli expectation
h1 = (0, 0)
h2 = (0, 0)
chi = {(h1, h2): 1}

ss = stabilizer.stabilizer_state("-ZZ","XX")
gss = GeneralStabilizerState(chi, ss.gs, ss.ps)

In [22]:
ss.expect(paulialg.paulis('IZ','-YX'))

array([0, 0])

In [23]:
gss.pauli_expect(paulialg.paulis('IZ','-YX'))

array([0.+0.j, 0.+0.j])